In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [3]:
gpu = torch.device('cuda:0')
model_id = "google/codegemma-1.1-2b"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map='auto', torch_dtype=torch.bfloat16)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


in oss file


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
# create sample input

In [12]:
import requests
sample = requests.get('https://raw.githubusercontent.com/pallets/flask/main/tests/test_basic.py').content.decode('utf-8')

In [48]:
def infer_batch(prompt, prompt_tokens=-1, max_new_tokens=128, top_p=0.9, temperature=0.1, num_return_sequences=1):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    if prompt_tokens < 0:
        prompt_tokens = len(inputs.input_ids[0])
    with torch.no_grad():
        generated_ids = model.generate(input_ids=inputs.input_ids[:,-prompt_tokens:],
                                       attention_mask=inputs.attention_mask[:,-prompt_tokens:],
                                       max_new_tokens=max_new_tokens, 
                                       #stopping_criteria=stopper, 
                                       top_p=top_p,
                                       temperature=temperature,
                                       do_sample=True,
                                       num_return_sequences=num_return_sequences)
    output = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    del generated_ids
    del inputs
    return output

In [52]:
import time
from tqdm import tqdm
import random


def last_tokens(prompt, tokens):
    return tokenizer.decode(tokenizer(prompt, return_tensors='pt').input_ids[:,-tokens:][0])

def benchmark(prompt, tokens_in, tokens_out, batch_sizes, n_iter):
    experiments = []
    results = []
    for num_in in tokens_in:
        for num_out in tokens_out:
            for batch_size in batch_sizes:
                for n in range(n_iter):
                    if batch_size * (num_in + num_out) <= LIMIT:
                        experiments.append((num_in, num_out, batch_size, n))
    print(f'{len(experiments)=}')
    
    random.shuffle(experiments)
    
    for num_in, num_out, batch_size, n in tqdm(experiments):
        print(f'{num_in=}, {num_out=}, {batch_size=}')
        input = last_tokens(prompt, num_in)
        torch.cuda.empty_cache()
        t = time.time()
        infer_batch(input, 
                    max_new_tokens=num_out, 
                    num_return_sequences=batch_size)
        t = time.time() - t
        results.append((num_in, num_out, batch_size, n, t))
    return results

In [62]:
LIMIT = 2048
device = gpu
bench_tokens_in = range(128, 1025, 128)
bench_tokens_out = range(64, 513, 64)
bench_batch_size = [1, 4, 16]
n_iter = 10
b = benchmark(sample, bench_tokens_in, bench_tokens_out, bench_batch_size, n_iter)

len(experiments)=760


  0%|          | 0/760 [00:00<?, ?it/s]

num_in=896, num_out=256, batch_size=1


  0%|          | 1/760 [00:02<31:27,  2.49s/it]

num_in=896, num_out=512, batch_size=1


...

100%|█████████▉| 758/760 [20:39<00:04,  2.00s/it]

num_in=384, num_out=256, batch_size=1


100%|█████████▉| 759/760 [20:41<00:02,  2.11s/it]

num_in=768, num_out=192, batch_size=1


100%|██████████| 760/760 [20:43<00:00,  1.64s/it]


In [73]:
from collections import defaultdict
import numpy as np

def tps(n_in, n_out, samples):
    return round(np.mean([n_out/s for s in samples]), 1)

def avg(n_in, n_out, samples):
    return round(np.mean(samples), 2)

def print_bench(b, bench_batch_size=bench_batch_size, bench_tokens_in=bench_tokens_in, bench_tokens_out=bench_tokens_out, aggregate=avg):
    b.sort()
    
    for current_size in bench_batch_size:
        
        print('-'*60)
        print(f'parallel generations: {current_size}')
    
        data = defaultdict(list)
        for n_in, n_out, b_size, _, timing in b:
            if b_size == current_size:
                data[(n_in, n_out)].append(timing)
                
        print('in\tout')
        line = '\t'
        for n_out in bench_tokens_out:
            line += str(n_out) + '\t'
        print(line)
        for n_in in bench_tokens_in:
            line = str(n_in) + '\t'
            for n_out in bench_tokens_out:
                samples = data.get((n_in, n_out), None)
                if samples:
                    line += str(aggregate(n_in, n_out, samples)) + '\t'
                else:
                    line += '--\t'
            print(line)



In [76]:
# generation times
print_bench(b, aggregate=avg)

------------------------------------------------------------
parallel generations: 1
in	out
	64	128	192	256	320	384	448	512	
128	0.02	0.02	0.02	0.02	0.02	0.02	0.02	0.02	
256	0.03	0.03	0.03	0.03	0.03	0.03	0.03	0.03	
384	0.61	1.2	1.8	2.4	3.0	3.57	4.17	4.79	
512	0.61	1.21	1.8	2.41	3.0	3.6	4.19	4.77	
640	0.62	1.22	1.81	2.41	3.0	3.61	4.2	4.8	
768	0.63	1.22	1.83	2.41	3.01	3.6	4.2	4.8	
896	0.63	1.23	1.83	2.42	2.91	3.61	4.03	4.56	
1024	0.46	0.76	0.77	1.0	0.65	2.19	3.38	2.44	
------------------------------------------------------------
parallel generations: 4
in	out
	64	128	192	256	320	384	448	512	
128	0.04	0.04	0.04	0.04	0.04	0.04	--	--	
256	0.06	0.06	0.06	0.06	--	--	--	--	
384	0.71	1.36	--	--	--	--	--	--	
512	--	--	--	--	--	--	--	--	
640	--	--	--	--	--	--	--	--	
768	--	--	--	--	--	--	--	--	
896	--	--	--	--	--	--	--	--	
1024	--	--	--	--	--	--	--	--	
------------------------------------------------------------
parallel generations: 16
in	out
	64	128	192	256	320	384	448	512	
128	--	--	--	--	--	-

In [77]:
# tokens per second
print_bench(b, aggregate=tps)

------------------------------------------------------------
parallel generations: 1
in	out
	64	128	192	256	320	384	448	512	
128	2889.7	5972.4	8747.7	11417.7	14215.9	17496.7	20387.6	22624.9	
256	2397.8	4712.5	7023.9	9402.5	11482.6	14220.7	16288.4	18979.4	
384	105.0	106.5	106.8	106.8	106.8	107.5	107.4	106.9	
512	104.1	105.8	106.6	106.4	106.5	106.7	107.0	107.4	
640	103.1	105.2	105.9	106.1	106.5	106.5	106.6	106.6	
768	101.8	104.6	105.1	106.2	106.3	106.6	106.8	106.7	
896	101.3	104.5	105.1	105.7	110.6	106.3	113.8	116.8	
1024	399.3	932.1	2000.3	2633.6	4386.1	2715.4	1606.2	4457.5	
------------------------------------------------------------
parallel generations: 4
in	out
	64	128	192	256	320	384	448	512	
128	1662.0	3326.5	5035.2	6537.5	8244.2	10053.3	--	--	
256	1092.6	2185.4	3289.4	4365.3	--	--	--	--	
384	89.8	94.0	--	--	--	--	--	--	
512	--	--	--	--	--	--	--	--	
640	--	--	--	--	--	--	--	--	
768	--	--	--	--	--	--	--	--	
896	--	--	--	--	--	--	--	--	
1024	--	--	--	--	--	--	--	--	
----------------

In [80]:
# median tokens per second
np.median([tps(n_in, n_out, [s]) for n_in, n_out, _, _, s in b])

106.95

In [81]:
# store results
import pickle
with open('llm-benchmark.pickle', 'wb') as fp:
    pickle.dump(b, fp)

In [69]:
# reload results
with open('llm-benchmark.pickle', 'rb') as fp:
    b = pickle.load(fp)

[(128, 64, 1, 0, 0.023736238479614258),
 (128, 64, 1, 1, 0.0209963321685791),
 (128, 64, 1, 2, 0.023659467697143555),
 (128, 64, 1, 3, 0.02104806900024414),
 (128, 64, 1, 4, 0.023738861083984375),
 (128, 64, 1, 5, 0.02096724510192871),
 (128, 64, 1, 6, 0.02392578125),
 (128, 64, 1, 7, 0.022480487823486328),
 (128, 64, 1, 8, 0.020773649215698242),
 (128, 64, 1, 9, 0.020944595336914062),
 (128, 64, 4, 0, 0.041047096252441406),
 (128, 64, 4, 1, 0.03752493858337402),
 (128, 64, 4, 2, 0.03747844696044922),
 (128, 64, 4, 3, 0.04098820686340332),
 (128, 64, 4, 4, 0.03745436668395996),
 (128, 64, 4, 5, 0.03761100769042969),
 (128, 64, 4, 6, 0.03762030601501465),
 (128, 64, 4, 7, 0.04076075553894043),
 (128, 64, 4, 8, 0.03754830360412598),
 (128, 64, 4, 9, 0.037644147872924805),
 (128, 128, 1, 0, 0.02371382713317871),
 (128, 128, 1, 1, 0.020860671997070312),
 (128, 128, 1, 2, 0.020993947982788086),
 (128, 128, 1, 3, 0.020980358123779297),
 (128, 128, 1, 4, 0.021054744720458984),
 (128, 128, 1, 